### Training RL Policies using L5Kit Closed-Loop Environment

This notebook describes how to train RL policies for self-driving using our gym-compatible closed-loop environment.

We will be using [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) algorithm as our reinforcement learning algorithm, as it not only demonstrates remarkable performance but it is also empirically easy to tune.

The PPO implementation in this notebook is based on [Stable Baselines3](https://github.com/DLR-RM/stable-baselines3) framework, a popular framework for training RL policies. Note that our environment is also compatible with [RLlib](https://docs.ray.io/en/latest/rllib.html), another popular frameworks for the same.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#@title Download L5 Sample Dataset and install L5Kit
import os
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !wget https://raw.githubusercontent.com/lyft/l5kit/master/examples/setup_notebook_colab.sh -q
    !sh ./setup_notebook_colab.sh
    os.environ["L5KIT_DATA_FOLDER"] = open("./dataset_dir.txt", "r").read().strip()
else:
    os.environ["L5KIT_DATA_FOLDER"] = "/home/pronton/rl/l5kit_dataset/"
    print("Not running in Google Colab.")

Not running in Google Colab.


In [ ]:
!pip install stable_baselines3
# !pip install -U "ray[default]"

In [2]:
import gym

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import get_linear_fn
from stable_baselines3.common.vec_env import SubprocVecEnv

from l5kit.configs import load_config_data
from l5kit.environment.feature_extractor import CustomFeatureExtractor
from l5kit.environment.callbacks import L5KitEvalCallback
from l5kit.environment.envs.l5_env import SimulationConfigGym

from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show

In [3]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable

# get environment config
env_config_path = '/home/pronton/rl/l5kit/examples/RL/gg colabs/gym_config.yaml'
cfg = load_config_data(env_config_path)

### Define Training and Evaluation Environments

**Training**: We will be training the PPO policy on episodes of length 32 time-steps. We will have 4 sub-processes (training environments) that will help to parallelize and speeden up episode rollouts. The *SimConfig* dataclass will define the parameters of the episode rollout: like length of episode rollout, whether to use log-replayed agents or simulated agents etc.

**Evaluation**: We will evaluate the performance of the PPO policy on the *entire* scene (~248 time-steps).

In [ ]:
# Train on episodes of length 32 time steps
train_eps_length = 32
train_envs = 4

# Evaluate on entire scene (~248 time steps)
eval_eps_length = None
eval_envs = 1

# make train env
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
env = make_vec_env("L5-CLE-v0", env_kwargs=env_kwargs, n_envs=train_envs,
                   vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

# make eval env
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, \
                   'return_info': True, 'train': False, 'sim_cfg': validation_sim_cfg}
eval_env = make_vec_env("L5-CLE-v0", env_kwargs=eval_env_kwargs, n_envs=eval_envs,
                        vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

### Define backbone feature extractor

The backbone feature extractor is shared between the policy and the value networks. The feature extractor *simple_gn* is composed of two convolutional networks followed by a fully connected layer, with ReLU activation. The feature extractor output is passed to both the policy and value networks composed of two fully connected layers with tanh activation (SB3 default).

We perform **group normalization** after every convolutional layer. Empirically, we found that group normalization performs far superior to batch normalization. This can be attributed to the fact that activation statistics change quickly in on-policy algorithms (PPO is on-policy) while batch-norm learnable parameters can be slow to update causing training issues.

In [ ]:
# A simple 2 Layer CNN architecture with group normalization
model_arch = 'simple_gn'
features_dim = 128

# Custom Feature Extractor backbone
policy_kwargs = {
    "features_extractor_class": CustomFeatureExtractor,
    "features_extractor_kwargs": {"features_dim": features_dim, "model_arch": model_arch},
    "normalize_images": False
}

### Clipping Schedule

We linearly decrease the value of the clipping parameter $\epsilon$ as the PPO training progress as it shows improved training stability

In [ ]:
# Clipping schedule of PPO epsilon parameter
start_val = 0.1
end_val = 0.01
training_progress_ratio = 1.0
clip_schedule = get_linear_fn(start_val, end_val, training_progress_ratio)

### Hyperparameters for PPO. 

For detailed description, refer https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/ppo/ppo.html#PPO

In [ ]:
lr = 3e-4
num_rollout_steps = 256
gamma = 0.8
gae_lambda = 0.9
n_epochs = 10
seed = 42
batch_size = 64
import datetime
tensorboard_log = 'tb_logs/'  + str(datetime.date.today()) + '/'

### Define the PPO Policy.

SB3 provides an easy interface to the define the PPO policy. Note: We do need to tweak appropriate hyperparameters and the custom policy backbone has been defined above.


In [ ]:
# define model
# model = PPO("MultiInputPolicy", env, policy_kwargs=policy_kwargs, verbose=1, n_steps=num_rollout_steps,
#             learning_rate=lr, gamma=gamma, tensorboard_log=tensorboard_log, n_epochs=n_epochs,
#             clip_range=clip_schedule, batch_size=batch_size, seed=seed, gae_lambda=gae_lambda)


In [ ]:

# model = SAC('MultiInputPolicy', env, policy_kwargs=policy_kwargs, verbose=1,  batch_size=batch_size,  learning_rate= lr,  gamma=gamma, tensorboard_log=tensorboard_log, buffer_size=10000,
#             learning_starts=batch_size) 
model = SAC("MultiInputPolicy", env, batch_size=batch_size, learning_rate= lr,
                    gamma=gamma,
                    buffer_size=10000,
                    tensorboard_log=tensorboard_log, 
                    train_freq= (1, "step"),
                    gradient_steps=1, # gradient 1 time per 1 episode
                    target_update_interval=200, # each ...(200) steps, we update target_net (load weight from policy_net)
                    learning_starts=batch_size,
                    policy_kwargs=policy_kwargs,
                    verbose=1,
                    seed=42,
                    use_sde=True,
                    sde_sample_freq=-1,
                    use_sde_at_warmup=True,
                    tau=.95, 
                    create_eval_env=True,
                    )

### Defining Callbacks

We can additionally define callbacks to save model checkpoints and evaluate models during training.

In [ ]:
callback_list = []

# Save Model Periodically
save_freq = 10000
save_path = '/content/drive/MyDrive/Colab Notebooks/logs/'
output_old = 'PPO'
output = 'SAC'
checkpoint_callback = CheckpointCallback(save_freq=(save_freq // train_envs), save_path=save_path, \
                                         name_prefix=output)
callback_list.append(checkpoint_callback)

# Eval Model Periodically
eval_freq = 10000
n_eval_episodes = 1
val_eval_callback = L5KitEvalCallback(eval_env, eval_freq=(eval_freq // train_envs), \
                                      n_eval_episodes=n_eval_episodes, n_eval_envs=eval_envs)
callback_list.append(val_eval_callback)


### Train

In [ ]:
outdir = '/content/drive/MyDrive/Colab Notebooks/sac_interrupted/'
model_name = 'SAC'
try:
    n_steps = 1000000
    model.learn(n_steps, callback=callback_list)
except:
    model.save(outdir + model_name)
    model.save_replay_buffer(outdir + model_name+ "_buffer")
    model.policy.save(outdir + model_name + "_policy")

**Voila!** We have a trained PPO policy! Train for larger number of steps for better accuracy. Typical RL algorithms require training atleast 1M steps for good convergence. You can visualize the quantitiative evaluation using tensorboard.

In [ ]:
# Visualize Tensorboard logs (!! run on local terminal !!)
# !tensorboard --logdir tb_log

### Visualize the episode from the environment

We can easily visualize the outputs obtained by rolling out episodes in the L5Kit using the Bokeh visualizer.

In [ ]:
model = SAC.load('/home/pronton/rl/l5kit/examples/RL/gg colabs/logs/SAC_640000_steps.zip', env = env
        #          , custom_objects = {
        #     "learning_rate": 0.0,
        #     "lr_schedule": lambda _: 0.0,
        #     "clip_range": lambda _: 0.0,
        # }
        )

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
rollout_env = gym.make("L5-CLE-v0", env_config_path=env_config_path, sim_cfg=rollout_sim_cfg, \
                       use_kinematic=True, train=False, return_info=True)

def rollout_episode(model, env, idx = 0):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.reset_scene_id = idx
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    while True:
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, info = env.step(action)
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out

# Rollout one episode
# sim_out = rollout_episode(model, rollout_env)
# Rollout 5 episodes
sim_outs =[]
for i in range(1):
    sim_outs.append(rollout_episode(model, rollout_env))

In [ ]:
# might change with different rasterizer
map_API = rollout_env.dataset.rasterizer.sem_rast.mapAPI

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs, map_API)

## Calculate the performance metrics from the episode outputs

We can also calculate the various quantitative metrics on the rolled out episode output. 

In [ ]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else L2DisplacementYawMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4)])
    print(table)


quantify_outputs(sim_outs)

In [ ]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else CLEMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE", "DRT", "CF", "CR", "CS", "PEGO"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    tot_drt = 0.0
    tot_cf = 0.0
    tot_cr = 0.0
    tot_cs = 0.0
    tot_p_ego = 0.0
    tot_a_ego = 0.0
    # print(scene_results[0])
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        drt_error = scene_metrics['distance_to_reference_trajectory'][-1]
        cf_error = scene_metrics['collision_front'][-1]
        cr_error = scene_metrics['collision_rear'][-1]
        cs_error = scene_metrics['collision_side'][-1]
        p_ego = scene_metrics['simulated_minus_recorded_ego_speed'][-1]
        # a_ego = scene_metrics['aggressive_ego'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4), round(drt_error.item(), 4), round(cf_error.item(), 4), round(cr_error.item(), 4), 
        round(cs_error.item(), 4), round(p_ego.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()
        tot_drt += drt_error.item()
        tot_cf += cf_error.item()
        tot_cr += cr_error.item()
        tot_cs += cs_error.item()
        tot_p_ego += p_ego.item()
        # tot_a_ego += a_ego.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    ave_drt = tot_drt / len(scene_results)
    ave_cf = tot_cf / len(scene_results)
    ave_cr = tot_cr / len(scene_results)
    ave_cs = tot_cs / len(scene_results)
    ave_p_ego = tot_p_ego / len(scene_results)
    # ave_a_ego = tot_a_ego / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4), round(ave_drt, 4), round(ave_cf, 4), round(ave_cr, 4), round(ave_cs, 4), round(ave_p_ego, 4)])
    print(table)


quantify_outputs(sim_outs)

In [ ]:
# model = SAC.load('/content/SAC_270000_steps.zip', env = env)
# n_steps = 1000000
# # model = PPO.load('./PPO_100000_steps.zip', env = env)
# model.learn(n_steps, callback=callback_list, reset_num_timesteps=False)